# Compute the readout fidelity of all qubits

In [ ]:
import numpy as np
from qibo.gates import M, X
from qibo.models import Circuit
from qiboconnection.api import API
from qiboconnection.connection import ConnectionConfiguration

api = API(ConnectionConfiguration(username="qat", api_key="meow"))
api.select_device_id(9)

We first generate and queue the circuits we need to execute:

In [ ]:
qubits = np.arange(5)
ids = []
for qubit in qubits:
    circuit0 = Circuit(5)
    circuit0.add(M(qubit))

    circuit1 = Circuit(5)
    circuit1.add(X(qubit))
    circuit1.add(M(qubit))

    ids.append(api.execute([circuit0, circuit1], nshots=1000))

print(f"The ids of the executed jobs are: {ids}")

Once the jobs have been executed, we can use the cell below to compute the readout fidelities:

In [ ]:

for qubit_idx, result_id in zip(qubits, ids):
    results0, results1 = api.get_results(result_id)

    clf_0 = (1 - results0["probabilities"]["0"] + results0["probabilities"]["1"]) / 2
    clf_1 = (1 - results1["probabilities"]["0"] + results1["probabilities"]["1"]) / 2
    fid_avg = (1 - clf_0 + clf_1) / 2
    print(f"### QUBIT: {qubit_idx} ###")
    print(f"Ground state: {results0}")
    print(f"Excited state: {results1}")
    print("Fidelity", fid_avg)
